# Find Wikidata ID for Wikisource Pages

This notebook finds the Wikidata ID for a given Wikisource URL.

In [3]:
import requests
import urllib.parse

In [4]:
def get_wikidata_id(wikisource_url):
    """
    Get Wikidata ID for a Wikisource page.
    Handles redirects and different apostrophe characters.
    """
    headers = {'User-Agent': 'WikidataBot/1.0 (Python requests)'}
    
    # Extract page title from URL
    parts = wikisource_url.split('/wiki/')
    if len(parts) < 2:
        return None
    
    page_title = urllib.parse.unquote(parts[1])
    
    # Determine the wiki site from URL (e.g., 'fr' from 'fr.wikisource.org')
    wiki_site = wikisource_url.split('//')[1].split('.')[0] + 'wikisource'
    api_base = f"https://{wikisource_url.split('//')[1].split('/')[0]}/w/api.php"
    
    # Step 1: Follow redirects to get the actual page title
    params = {
        'action': 'query',
        'titles': page_title,
        'redirects': '',
        'prop': 'pageprops',
        'format': 'json'
    }
    
    try:
        r = requests.get(api_base, params=params, headers=headers, timeout=15)
        data = r.json()
        
        # Check for redirects
        redirects = data.get('query', {}).get('redirects', [])
        if redirects:
            page_title = redirects[-1]['to']  # Get final redirect target
        
        # Check if wikibase_item is in pageprops
        pages = data.get('query', {}).get('pages', {})
        for page_id, page_data in pages.items():
            if page_id != '-1':
                wikibase_item = page_data.get('pageprops', {}).get('wikibase_item')
                if wikibase_item:
                    return wikibase_item
    except Exception as e:
        print(f"Wikisource API error: {e}")
    
    # Step 2: Query Wikidata directly using the (possibly redirected) title
    try:
        wd_api_url = "https://www.wikidata.org/w/api.php"
        params = {
            'action': 'wbgetentities',
            'sites': wiki_site,
            'titles': page_title,
            'format': 'json'
        }
        
        r = requests.get(wd_api_url, params=params, headers=headers, timeout=15)
        data = r.json()
        
        entities = data.get('entities', {})
        for entity_id, entity_data in entities.items():
            if entity_id != '-1' and not entity_id.startswith('-'):
                return entity_id
    except Exception as e:
        print(f"Wikidata API error: {e}")
    
    return None

In [13]:
# Test with L'Internationale
url = "https://fr.wikisource.org/wiki/L'Internationale"
url = 'https://fr.wikisource.org/wiki/L%E2%80%99Insurg%C3%A9_(Vall%C3%A8s)'
wikidata_id = get_wikidata_id(url)
print(f"URL: {url}")
print(f"Wikidata ID: {wikidata_id}")
if wikidata_id:
    print(f"Wikidata URL: https://www.wikidata.org/wiki/{wikidata_id}")

URL: https://fr.wikisource.org/wiki/L%E2%80%99Insurg%C3%A9_(Vall%C3%A8s)
Wikidata ID: Q19180261
Wikidata URL: https://www.wikidata.org/wiki/Q19180261


## Results

The Wikidata ID for "L'Internationale" on French Wikisource is **Q55989489**.

**Note:** The original URL uses a straight apostrophe (`'`), but Wikisource redirects to a page with a curly apostrophe (`'`). The improved function handles this by following redirects.

In [34]:
# only get the root one

import pandas as pd
import urllib.parse

data = pd.read_csv('wikisource_pages_with_urls.csv')

content = data[data['wikisource_url']=='https://fr.wikisource.org/wiki/Serment_d’Hippocrate_(Daremberg)']['content'].iloc[0]

# Save to text file
with open('hippocrate_serment_content.txt', 'w', encoding='utf-8') as f:
    f.write(content)

In [35]:
data[data['wikisource_url']=='https://fr.wikisource.org/wiki/Serment_d’Hippocrate_(Daremberg)']['content'].iloc[0]

'{{TextQuality|100%}}\n{{Voir traductions|\'\'[[Serment (Hippocrate)|Le Serment d’Hippocrate]]\'\'}}\n<pages index="Hippocrate - Oeuvres choisies, trad Daremberg, 1844.djvu" from=43 to=426 exclude="46-420" tosection=serment header=1 titre="Le Serment" displayed_to=3/>\n\n[[Catégorie:Littérature grecque]]\n[[Catégorie:Bon pour export]]\n[[Catégorie:Médecine]]'

In [40]:
print(data[data['title']=='Voyage sur le Mississipi']['content'].iloc[0])

{{TextQuality|Textes validés}}

<pages index="Revue des Deux Mondes - 1833 - tome 1.djvu" from=531 to=550  header=1 auteur="[[Auteur:Eugène Ney|Eugène Ney]]" prev="" next="" />

[[Catégorie:Articles de la Revue des Deux Mondes]]
[[Catégorie:Explorations et voyages aux États-Unis]]
[[Catégorie:Articles par Auteur]]
[[Catégorie:Articles de 1833]]


In [44]:
# Récupérer le contenu
content = data[data['title']=='Voyage sur le Mississipi']['content'].iloc[0]

# Vérifier la longueur totale
print(f"Longueur totale du contenu: {len(content)} caractères")

# Sauvegarder dans un fichier texte pour tout voir
with open('voyage_mississipi_content.txt', 'w', encoding='utf-8') as f:
    f.write(content)

print("Contenu sauvegardé dans 'voyage_mississipi_content.txt'")

# Si vous voulez voir les premiers X caractères
print("\nPremiers 2000 caractères:")
print(content[:2000])

# Ou voir les derniers caractères
print("\nDerniers 500 caractères:")
print(content[-500:])

Longueur totale du contenu: 346 caractères
Contenu sauvegardé dans 'voyage_mississipi_content.txt'

Premiers 2000 caractères:
{{TextQuality|Textes validés}}

<pages index="Revue des Deux Mondes - 1833 - tome 1.djvu" from=531 to=550  header=1 auteur="[[Auteur:Eugène Ney|Eugène Ney]]" prev="" next="" />

[[Catégorie:Articles de la Revue des Deux Mondes]]
[[Catégorie:Explorations et voyages aux États-Unis]]
[[Catégorie:Articles par Auteur]]
[[Catégorie:Articles de 1833]]

Derniers 500 caractères:
{{TextQuality|Textes validés}}

<pages index="Revue des Deux Mondes - 1833 - tome 1.djvu" from=531 to=550  header=1 auteur="[[Auteur:Eugène Ney|Eugène Ney]]" prev="" next="" />

[[Catégorie:Articles de la Revue des Deux Mondes]]
[[Catégorie:Explorations et voyages aux États-Unis]]
[[Catégorie:Articles par Auteur]]
[[Catégorie:Articles de 1833]]


## Exratc main information

In [43]:
data[data['title']=='Voyage sur le Mississipi']['wikisource_url'].iloc[0]

'https://fr.wikisource.org/wiki/Voyage_sur_le_Mississipi'

In [ ]:



# keep roots

# Function to extract root URL (remove everything after the last slash if it looks like a chapter/page number)
def get_root_url(url):
    """
    Extract root URL by removing trailing parts like /1, /2, /Chapitre_1, etc.
    Keeps namespace prefixes like Page:, MediaWiki:, etc.
    """
    if pd.isna(url):
        return None
    
    # Parse the URL
    parts = url.split('/wiki/')
    if len(parts) < 2:
        return url
    
    base = parts[0] + '/wiki/'
    path = parts[1]
    
    # Split by '/' to check if there are sub-pages
    path_parts = path.split('/')
    
    # If there's only one part, it's already a root
    if len(path_parts) == 1:
        return url
    
    # Check if the last part looks like a chapter/section (number or roman numeral pattern)
    last_part = urllib.parse.unquote(path_parts[-1])
    
    # Keep only the first part (root) if last part seems like a sub-page
    # This handles cases like "Book_Title/1", "Book_Title/Chapter_1", etc.
    root_path = path_parts[0]
    
    return base + root_path

# Apply the function
data['root_url'] = data['wikisource_url'].apply(get_root_url)

# Group by root URL
grouped = data.groupby('root_url').agg({
    'title': 'first',  # Take first title as representative
    'page_id': 'count',  # Count number of pages
    'wikisource_url': lambda x: list(x)  # Keep all original URLs
}).rename(columns={'page_id': 'num_pages', 'wikisource_url': 'all_urls'})

# Reset index to make root_url a column
grouped = grouped.reset_index()

# Sort by number of pages (descending)
grouped = grouped.sort_values('num_pages', ascending=False)

# Also create a version with just the roots (no sub-pages)
roots_only = data[data['wikisource_url'] == data['root_url']].copy()
print(f"\nTotal root pages (no sub-pages): {len(roots_only)}")

roots_data = roots_only.copy()


Total root pages (no sub-pages): 5693


In [22]:
test_urls = roots_data['wikisource_url'].sample(30, random_state=42).dropna().unique()


In [ ]:
print("Testing multiple ROOT URLs:\n")

# Store results in a list
results = []

for url in test_urls:
    wikidata_id = get_wikidata_id(url)
    title = url.split('/wiki/')[-1]
    decoded_title = urllib.parse.unquote(title)
    
    print(f"• {decoded_title}")
    print(f"  Wikidata ID: {wikidata_id}")
    if wikidata_id:
        wikidata_url = f"https://www.wikidata.org/wiki/{wikidata_id}"
        print(f"  URL: {wikidata_url}")
    else:
        wikidata_url = None
    print()
    
    # Append to results
    results.append({
        'wikisource_url': url,
        'title': decoded_title,
        'wikidata_id': wikidata_id,
        'wikidata_url': wikidata_url
    })

# Create DataFrame
results_df = pd.DataFrame(results)


In [28]:
results_df

,wikisource_url,title,wikidata_id,wikidata_url
0,https://fr.wikisource.org/wiki/Les_Oiseaux,Les_Oiseaux,Q27079396,https://www.wikidata.org/wiki/Q27079396
1,https://fr.wikisource.org/wiki/La_Grève_des_Mères,La_Grève_des_Mères,Q19183883,https://www.wikidata.org/wiki/Q19183883
2,https://fr.wikisource.org/wiki/Voyage_dans_la_...,"Voyage_dans_la_Tartarie,_l’Afghanistan_et_l’In...",Q19239775,https://www.wikidata.org/wiki/Q19239775
3,https://fr.wikisource.org/wiki/Prière_de_l’ind...,Prière_de_l’indigent,Q19225186,https://www.wikidata.org/wiki/Q19225186
4,https://fr.wikisource.org/wiki/Auteur:Tristan_...,Auteur:Tristan_Corbière,Q316536,https://www.wikidata.org/wiki/Q316536
5,https://fr.wikisource.org/wiki/Serment_d’Hippo...,Serment_d’Hippocrate_(Daremberg),None,None
6,https://fr.wikisource.org/wiki/Catégorie:Auteu...,Catégorie:Auteurs_de_l'Antiquité,Q7437391,https://www.wikidata.org/wiki/Q7437391
7,https://fr.wikisource.org/wiki/Catégorie:Histo...,Catégorie:Historiens,Q6395208,https://www.wikidata.org/wiki/Q6395208
8,https://fr.wikisource.org/wiki/L'Aigle_et_l'Es...,L'Aigle_et_l'Escarbot,None,None
9,https://fr.wikisource.org/wiki/Auteur:Francis_...,Auteur:Francis_Charmes,Q2341785,https://www.wikidata.org/wiki/Q2341785


In [27]:
df_final =results_df[results_df['wikidata_id'].isna()]
list(df_final['wikisource_url'])

['https://fr.wikisource.org/wiki/Serment_d’Hippocrate_(Daremberg)',
 "https://fr.wikisource.org/wiki/L'Aigle_et_l'Escarbot",
 "https://fr.wikisource.org/wiki/Les_Derniers_jours_d'Emmanuel_Kant",
 'https://fr.wikisource.org/wiki/Un_prince_de_la_bohème']